In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np
from torch.distributions.normal import Normal
import warnings
warnings.filterwarnings('ignore')

# device, cuda if available else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from utils import *
from train import *
from data_utils import *
from modules.deformable_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

# **1 - Get The Data :**

In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/vae_mlp.yaml', 'r') as stream:
    vae_mlp_args  = yaml.load(stream,Loader=yaml.Loader)

In [5]:
%%time
# Getting the data
# dataset for non lung
non_lung_dataset = TranscriptomicVectorsDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = vae_mlp_args["batch_size_pt"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_pt"], test_proportion = 0.0)

CPU times: user 13.1 s, sys: 216 ms, total: 13.3 s
Wall time: 12.7 s


In [36]:
%%time
# Getting the data
# dataset for non lung
non_lung_dataloader_train_splitted, non_lung_dataloader_validation_splitted = get_data_loaders(non_lung_dataset, batch_size_train = vae_mlp_args["batch_size_pt"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_pt"])

CPU times: user 667 ms, sys: 3.55 ms, total: 671 ms
Wall time: 668 ms


In [8]:
%%time
# Getting the data
# dataset for lung
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = vae_mlp_args["batch_size_ft"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_ft"])

CPU times: user 11.4 s, sys: 115 ms, total: 11.5 s
Wall time: 11.4 s


# **2 - Define the network's architecture :**
* Encoder and Decoder and then the full network (VAE) :

## 2-1- Encoder and Decoder with linear layers :

In [9]:
class Encoder(nn.Module):
    """
    VAE Encoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_input, hidden)
        # generate covariance matrix (hyp : diagonal)
        self.sigma = nn.Linear(hidden, dim_z)
        # genrate mean vector 
        self.mu = nn.Linear(hidden, dim_z)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        #we infer log(sigma**2) for stability reasons
        log_sigma_squared = self.sigma(lin1) 
        mu = self.mu(lin1)
        return mu, log_sigma_squared

In [10]:
class Decoder(nn.Module):
    """
    VAE Decoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_z, hidden)
        self.lin2 = nn.Linear(hidden, dim_input)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        x = torch.sigmoid(self.lin2(lin1))
        return x  

In [11]:
class VAE(nn.Module):
    """
    VAE with linear layers based Encoder and Decoder
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        # encoder
        self.encoder = Encoder(dim_input, hidden, dim_z).to(device)
        # decoder
        self.decoder = Decoder(dim_input, hidden, dim_z).to(device)
        
    def forward(self, x):
        mu, log_sigma_squared = self.encoder(x)
        z = self.get_codes(mu, log_sigma_squared)
        return self.decoder(z), mu, log_sigma_squared

    def get_codes(self, mu, log_sigma_squared):
        # input is log(sigma**2)
        # log(sigma**2) = 2 * log(sigma)
        # log(sigma) = (1/2)*log(sigma**2)
        # sigma = exp((1/2)*log(sigma**2))
        sigma = torch.exp((1/2) * log_sigma_squared)
        # reparametrisation trick
        eps = torch.empty_like(sigma).normal_().to(device)
        return eps.mul(sigma).add_(mu)
    def sample_data(self,z):
        return self.decoder(z)

In [22]:
def variational_lower_bound(oringinal, reconstructed, mu, log_sigma_squared):
    reconstructed = reconstructed
    oringinal = oringinal
    reconstruction_term = F.binary_cross_entropy(reconstructed, oringinal, reduction='sum')
    kl_div = - (1/2) * torch.sum(1 + log_sigma_squared - mu.pow(2) - log_sigma_squared.exp())
    return reconstruction_term + kl_div

In [45]:
# dimension of the latent variable z
dim_z = 300
# dimnesion of the generator's output/discriminator's input (an mnist image)
dim_input = len(non_lung_dataset[0][0])
# dimension of hidden layers
hidden = 256

In [46]:
vae = VAE(dim_input, hidden, dim_z).to(device).double()
print(vae)

VAE(
  (encoder): Encoder(
    (lin1): Linear(in_features=7509, out_features=256, bias=True)
    (sigma): Linear(in_features=256, out_features=300, bias=True)
    (mu): Linear(in_features=256, out_features=300, bias=True)
  )
  (decoder): Decoder(
    (lin1): Linear(in_features=300, out_features=256, bias=True)
    (lin2): Linear(in_features=256, out_features=7509, bias=True)
  )
)


# **3 - TRAINIG VAE WITH LINEAR BASED NETWORKS :**

In [47]:
# parameters
epochs = 50

# optimisation
lr = 0.001

# optimizer
optim_vae = optim.Adam(vae.parameters())

# noise to display samples during training
noise = torch.randn(vae_mlp_args["batch_size_pt"], dim_z, device=device)

In [48]:
# per epoch logging containers
v_lower_bound = []
image_and_reconstruction = []
sampled_images = []
for epoch in range(epochs):
    print("EPOCH : ",epoch)
    # per gradient step logging containers
    epoch_v_lower_bound = []
    
    for i, (vectors, labels) in enumerate(non_lung_dataloader_train):
        
        # perform a gradient step
        
        # flatten and push to device
        vectors = vectors.to(device).double()
        
        # run vae
        reconstruction, mu, log_sigma_squared = vae(vectors)

        # loss
        loss = variational_lower_bound(vectors, reconstruction, mu, log_sigma_squared)
        
        # backpropagation
        optim_vae.zero_grad()
        loss.backward()
        optim_vae.step()
        
        # logging
        epoch_v_lower_bound.append(loss.item())
        
        # display stats and an image during training
        """if i % 300 == 0 :
            print("BATCH :",i)
            print("| v_lower_bound: ", epoch_v_lower_bound[i])"""

    # per epoch logging
    v_lower_bound.append(sum(epoch_v_lower_bound) / len(epoch_v_lower_bound))

    # verbose every epoch
    print("| v_lower_bound: ", v_lower_bound[-1])

EPOCH :  0
| v_lower_bound:  994897.3243300397
EPOCH :  1
| v_lower_bound:  -12497242.013493897
EPOCH :  2
| v_lower_bound:  335572127.62069786
EPOCH :  3
| v_lower_bound:  -16548938.055012707
EPOCH :  4
| v_lower_bound:  -17113857.418690365
EPOCH :  5
| v_lower_bound:  -17732402.940693483
EPOCH :  6
| v_lower_bound:  2148981448064933.0
EPOCH :  7
| v_lower_bound:  -15431018.4117085
EPOCH :  8
| v_lower_bound:  -18823157.252085906
EPOCH :  9
| v_lower_bound:  -18807991.941802025
EPOCH :  10
| v_lower_bound:  -18321394.696602125
EPOCH :  11
| v_lower_bound:  -19279725.073079165
EPOCH :  12
| v_lower_bound:  -10518098.099455036
EPOCH :  13
| v_lower_bound:  -20272456.488140173
EPOCH :  14
| v_lower_bound:  -20164339.829643212
EPOCH :  15
| v_lower_bound:  -19262798.516282152
EPOCH :  16
| v_lower_bound:  -19988659.272980485
EPOCH :  17
| v_lower_bound:  7928403814689.964
EPOCH :  18
| v_lower_bound:  802417895498.5443
EPOCH :  19
| v_lower_bound:  -20808091.334914356
EPOCH :  20
| v_lowe

In [50]:
import copy
class MLP(nn.Module):
    """
    Implementation of A fully connected feed forward neural network
    """
    def __init__(self, input_size, output_size=2, layers = [160, 100], bias=True, dropout=0.8):
        """
        Args:
            - input_size: 
            - output_size: 
            - layers : dimensions des layers du MLP
            - bias : si True, ajouter un biais au module Linear
        """
        super(MLP, self).__init__()
        self.layers = layers
        self.output_size = output_size
        if len(layers)>0:
            self.fc = [nn.Linear(input_size, self.layers[0],bias=bias)]
            for i in range(len(self.layers)-1):
                self.fc.append(nn.Linear(self.layers[i], self.layers[i+1],bias=bias))
                self.fc.append(nn.BatchNorm1d(self.layers[i+1]))
                self.fc.append(nn.ReLU(inplace=True))
                self.fc.append(nn.Dropout(dropout))
            self.fc.append(nn.Linear(self.layers[-1], output_size,bias=bias))
        else:
            self.fc = [nn.Linear(input_size, output_size,bias=bias)]
        self.linear = nn.Sequential(*self.fc)
    
    def forward(self, x):
        return self.linear(x)
    
def fine_tune_mlp(pretrained, freeze = False):
    """Creates a model from a pre-trained MLP model using Transfer Learning"""
    """
    Args:
        - pretrained : (nn.Module) pre-trained MLP model to be used.
        - freeze : (bool) if True, params in hidden layers will be freezed.
    """
    model = copy.deepcopy(pretrained)
    if freeze:
        for i in range(len(model.fc)-1) :
            for param in model.fc[i].parameters():
                param.requires_grad = False
    model.fc[len(model.fc)-1] = nn.Linear(model.fc[len(model.fc)-5].out_features, 2,bias=True)
    return model

# Pre-Training : 

In [51]:
# network
net = MLP(dim_z).to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**vae_mlp_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_vae_checkpt.pt')
# Tensorboard summary writer
if vae_mlp_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train_splitted, non_lung_dataloader_validation_splitted, vae_mlp_args['epochs'], writer=writer, embedding_computer=vae)
if not vae_mlp_args['no_tensorboard']:
    writer.close()

Epoch 1/50:  17%|█▋        | 7/41 [00:00<00:00, 67.60it/s, loss=2.8878e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 2/50:  17%|█▋        | 7/41 [00:00<00:00, 68.84it/s, loss=1.3300e-01]

Epoch 1/50, Train Loss: 1.1003e-01, Test Loss: 0.1265
Epoch 1/50, Train Accuracy: 69.85%, Test Accuracy: 65.39%
Epoch 1/50, Train AUC: 69.86%, Test AUC: 66.54%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.65      0.77      2249
      Cancer       0.20      0.68      0.31       294

    accuracy                           0.65      2543
   macro avg       0.57      0.67      0.54      2543
weighted avg       0.85      0.65      0.72      2543



Epoch 3/50:  17%|█▋        | 7/41 [00:00<00:00, 68.73it/s, loss=1.3990e-01]

...

Epoch 48/50, Train Loss: 6.9104e-02, Test Loss: 0.1027
Epoch 48/50, Train Accuracy: 77.51%, Test Accuracy: 70.02%
Epoch 48/50, Train AUC: 77.48%, Test AUC: 66.48%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.71      0.81      2249
      Cancer       0.22      0.62      0.32       294

    accuracy                           0.70      2543
   macro avg       0.58      0.66      0.57      2543
weighted avg       0.85      0.70      0.75      2543



Epoch 50/50:  20%|█▉        | 8/41 [00:00<00:00, 70.21it/s, loss=9.8622e-02]

Epoch 49/50, Train Loss: 6.8351e-02, Test Loss: 0.1046
Epoch 49/50, Train Accuracy: 77.51%, Test Accuracy: 69.54%
Epoch 49/50, Train AUC: 77.51%, Test AUC: 66.95%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.70      0.80      2249
      Cancer       0.22      0.64      0.33       294

    accuracy                           0.70      2543
   macro avg       0.58      0.67      0.56      2543
weighted avg       0.85      0.70      0.75      2543



Epoch 50/50: 100%|██████████| 41/41 [00:00<00:00, 71.32it/s, loss=6.7216e-02]


Epoch 50/50, Train Loss: 6.5808e-02, Test Loss: 0.0907
Epoch 50/50, Train Accuracy: 78.40%, Test Accuracy: 70.32%
Epoch 50/50, Train AUC: 78.20%, Test AUC: 65.92%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.72      0.81      2249
      Cancer       0.22      0.60      0.32       294

    accuracy                           0.70      2543
   macro avg       0.57      0.66      0.56      2543
weighted avg       0.85      0.70      0.75      2543


Finished.
Best validation AUC: 6.9075e-01
With accuracy: 0.7067989856004715


# Fine-Tuning : 

In [52]:
# Load the pretrained Model
net =  MLP(dim_z).to(device).double()
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_vae_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

In [53]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_mlp(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_ft'])
savepath = Path('models_finetuned/fine_tuned_mlp_vae_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, vae_mlp_args['epochs'], embedding_computer=vae)

Epoch 2/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.3999e-01]

Training on GPU 

Epoch 1/50, Train Loss: 1.4167e-01, Test Loss: 0.1371
Epoch 1/50, Train Accuracy: 62.13%, Test Accuracy: 62.50%
Epoch 1/50, Train AUC: 62.31%, Test AUC: 59.23%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.68      0.79       260
      Cancer       0.12      0.50      0.19        22

    accuracy                           0.67       282
   macro avg       0.53      0.59      0.49       282
weighted avg       0.88      0.67      0.75       282



Epoch 3/50: 100%|██████████| 5/5 [00:00<00:00, 83.13it/s, loss=1.4938e-01]


Epoch 2/50, Train Loss: 1.1801e-01, Test Loss: 0.1595
Epoch 2/50, Train Accuracy: 69.60%, Test Accuracy: 63.26%
Epoch 2/50, Train AUC: 68.93%, Test AUC: 61.12%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.95      0.68      0.79       260
      Cancer       0.12      0.55      0.20        22

    accuracy                           0.67       282
   macro avg       0.54      0.61      0.50       282
weighted avg       0.88      0.67      0.74       282

Epoch 3/50, Train Loss: 1.0872e-01, Test Loss: 0.1373
Epoch 3/50, Train Accuracy: 70.99%, Test Accuracy: 69.19%
Epoch 3/50, Train AUC: 71.07%, Test AUC: 61.15%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.72      0.82       260
      Cancer       0.13      0.50      0.21        22

    accuracy                           0.71       282
   macro avg       0.54      0.61      0.51       282
weighted 

Epoch 6/50:   0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 46/50, Train Loss: 2.0752e-02, Test Loss: 0.0486
Epoch 46/50, Train Accuracy: 89.53%, Test Accuracy: 80.85%
Epoch 46/50, Train AUC: 90.18%, Test AUC: 59.18%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.87      0.90       260
      Cancer       0.17      0.32      0.22        22

    accuracy                           0.82       282
   macro avg       0.55      0.59      0.56       282
weighted avg       0.88      0.82      0.85       282

Epoch 47/50, Train Loss: 2.2930e-02, Test Loss: 0.0523
Epoch 47/50, Train Accuracy: 85.96%, Test Accuracy: 79.29%
Epoch 47/50, Train AUC: 86.40%, Test AUC: 58.02%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.84      0.89       260
      Cancer       0.15      0.32      0.20        22

    accuracy                           0.80       282
   macro avg       0.54      0.58      0.54       282
wei

Epoch 50/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=2.2218e-02]

Epoch 48/50, Train Loss: 1.5847e-02, Test Loss: 0.0592
Epoch 48/50, Train Accuracy: 90.25%, Test Accuracy: 78.02%
Epoch 48/50, Train AUC: 89.82%, Test AUC: 60.68%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.85      0.89       260
      Cancer       0.17      0.36      0.23        22

    accuracy                           0.81       282
   macro avg       0.56      0.61      0.56       282
weighted avg       0.88      0.81      0.84       282

Epoch 49/50, Train Loss: 2.0496e-02, Test Loss: 0.0670
Epoch 49/50, Train Accuracy: 87.78%, Test Accuracy: 81.87%
Epoch 49/50, Train AUC: 88.66%, Test AUC: 63.34%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.86      0.90       260
      Cancer       0.20      0.41      0.26        22

    accuracy                           0.82       282
   macro avg       0.57      0.63      0.58       282
wei

Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 82.49it/s, loss=3.1727e-02]


Epoch 50/50, Train Loss: 1.6467e-02, Test Loss: 0.0714
Epoch 50/50, Train Accuracy: 89.25%, Test Accuracy: 81.37%
Epoch 50/50, Train AUC: 89.71%, Test AUC: 65.80%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.95      0.86      0.90       260
      Cancer       0.22      0.45      0.29        22

    accuracy                           0.83       282
   macro avg       0.58      0.66      0.60       282
weighted avg       0.89      0.83      0.86       282


Finished.
Best validation AUC: 6.7115e-01
With accuracy: 0.7930689056714376
